# Preparations

Here I'll install library / define some constants / load/read dataset

In [1]:
!git clone https://github.com/alex4321/keras_elmo_bert

fatal: destination path 'keras_elmo_bert' already exists and is not an empty directory.


In [2]:
import os

os.chdir('/content/keras_elmo_bert')
!python3 setup.py install --force > /dev/null &2>1

zip_safe flag not set; analyzing archive contents...


In [3]:
%matplotlib inline
import zipfile
import numpy as np
import pandas as pd
import tensorflow as tf
from tqdm import tqdm_notebook as tqdm
from keras import backend as K
from keras.models import Model, load_model
from keras.layers import Activation, Conv1D, Dense, Dropout, Embedding, Flatten, GlobalMaxPool1D, LSTM, \
                         Input, MaxPooling1D, GlobalAveragePooling1D, multiply, concatenate, Reshape

sess = tf.Session()

Using TensorFlow backend.


In [0]:
MAX_SEQ_LENGTH = 256
DATASET_URL = "https://bitbucket.org/alexander_pozharskii/acl-imdb-concatenated/raw/48e80a99d5dbc24740597ad0df5274ef1dcb929b/aclImdb.zip"
RANDOM_STATE = 42

In [5]:
file = tf.keras.utils.get_file(fname="aclImdb_v1.zip",
                               origin=DATASET_URL,
                               extract=False)
with zipfile.ZipFile(file) as dataset:
  train_df = pd.read_csv(dataset.open('train.csv'))
  test_df = pd.read_csv(dataset.open('test.csv'))
train_df = train_df.loc[train_df['sentiment'] != 'unsup']
train_df['sentiment'] = (train_df['sentiment'] == 'pos').astype(np.int32)
test_df['sentiment'] = (test_df['sentiment'] == 'pos').astype(np.int32)
train_df['text'] = train_df['text'].str.lower()
test_df['text'] = test_df['text'].str.lower()
train_df = train_df.sample(len(train_df), random_state=RANDOM_STATE)
test_df = test_df.sample(len(test_df), random_state=RANDOM_STATE)

train_df.head()

,text,url,sentiment
6868,"silent night, deadly night 5 is the very last ...",http://www.imdb.com/title/tt0120793/usercommen...,0
24016,the idea ia a very short film with a lot of in...,http://www.imdb.com/title/tt0076770/usercommen...,1
9668,"for me, this movie just seemed to fall on its ...",http://www.imdb.com/title/tt0156524/usercommen...,0
13640,was this based on a comic-book? a video-game? ...,http://www.imdb.com/title/tt0102951/usercommen...,1
14018,caution: may contain spoilers...<br /><br />i'...,http://www.imdb.com/title/tt0065854/usercommen...,1


# ELMO

Below you'll ELMO-based classifier fine-tuning process.

In [6]:
from keras_elmo_bert import ElmoTokenizer, ElmoLayer

ELMO_PATH = "https://tfhub.dev/google/elmo/2"
ELMO_BATCH_SIZE = 64

W0609 09:05:39.433315 139943015159680 __init__.py:56] Some hub symbols are not available because TensorFlow version is less than 1.14


## Tokenization

In [0]:
tokenizer = ElmoTokenizer(MAX_SEQ_LENGTH)

In [0]:
train_tokens = tokenizer.predict(train_df['text'].tolist())
train_label = train_df['sentiment'].tolist()

test_tokens = tokenizer.predict(test_df['text'].tolist())
test_label = test_df['sentiment'].tolist()

## Model

In [0]:
def build_model_elmo(max_seq_length): 
    input_tokens = Input(shape=(max_seq_length,), dtype="string")
    bert_output = ElmoLayer(trainable=True, tf_hub=ELMO_PATH, output_representation='default')(input_tokens)
    
    dense = Dense(256, activation='relu')(bert_output)
    pred = Dense(1, activation='sigmoid')(dense)
    
    model = Model(inputs=input_tokens, outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model

def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)

In [10]:
elmo_model = build_model_elmo(MAX_SEQ_LENGTH)
initialize_vars(sess)

Instructions for updating:
Colocations handled automatically by placer.


W0609 09:05:46.382684 139943015159680 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0609 09:05:47.150115 139943015159680 saver.py:1483] Saver not created because there are no variables in the graph to restore


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 256)               0         
_________________________________________________________________
elmo_layer_1 (ElmoLayer)     (None, 1024)              93600852  
_________________________________________________________________
dense_1 (Dense)              (None, 256)               262400    
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 93,863,509
Trainable params: 262,661
Non-trainable params: 93,600,848
_________________________________________________________________


In [11]:
elmo_model.trainable_weights

[<tf.Variable 'elmo_layer_1_module/aggregation/weights:0' shape=(3,) dtype=float32>,
 <tf.Variable 'elmo_layer_1_module/aggregation/scaling:0' shape=() dtype=float32>,
 <tf.Variable 'dense_1/kernel:0' shape=(1024, 256) dtype=float32_ref>,
 <tf.Variable 'dense_1/bias:0' shape=(256,) dtype=float32_ref>,
 <tf.Variable 'dense_2/kernel:0' shape=(256, 1) dtype=float32_ref>,
 <tf.Variable 'dense_2/bias:0' shape=(1,) dtype=float32_ref>]

In [0]:
# Keras fails with "'str' object has no attribute 'ndim'"
# if I passed strings directly without wrapping to Tensor
# at least some versions
def elmo_iter(tokens, labels, batch_size):
    assert len(tokens) == len(labels)
    batch_count = int(np.ceil(len(tokens) / float(batch_size)))
    
    def _iter():
        while True:
            for i in range(batch_count):
                batch_tokens = tokens[i * batch_size : (i + 1) * batch_size]
                batch_labels = labels[i * batch_size : (i + 1) * batch_size]
                yield np.array(batch_tokens), np.array(batch_labels)
    
    return _iter(), batch_count

In [13]:
train_gen, train_steps = elmo_iter(train_tokens, train_label, batch_size=ELMO_BATCH_SIZE)
test_gen, test_steps = elmo_iter(test_tokens, test_label, batch_size=ELMO_BATCH_SIZE)
elmo_model.fit_generator(train_gen, train_steps,
                         validation_data=test_gen,
                         validation_steps=test_steps,
                         epochs=1,
                         verbose=True)

Instructions for updating:
Use tf.cast instead.


W0609 09:05:47.930455 139943015159680 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Epoch 1/1
391/391 [==============================] - 1348s 3s/step - loss: 0.4312 - acc: 0.7981 - val_loss: 0.3826 - val_acc: 0.8276


In [0]:
elmo_model.save('elmo.h5')

# BERT

In [0]:
from keras_elmo_bert import BertTokenizer, BertLayer

BERT_PATH = "https://tfhub.dev/google/bert_uncased_L-12_H-768_A-12/1"

## Tokenization

In [16]:
tokenizer = BertTokenizer(BERT_PATH, MAX_SEQ_LENGTH)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0609 09:28:22.385494 139943015159680 saver.py:1483] Saver not created because there are no variables in the graph to restore


In [17]:
# # Create datasets (Only take up to max_seq_length words for memory)
train_text = train_df['text'].tolist()
train_text = [' '.join(t.split()[0:MAX_SEQ_LENGTH]) for t in train_text]
train_text = np.array(train_text, dtype=object)
train_label = train_df['sentiment'].tolist()

test_text = test_df['text'].tolist()
test_text = [' '.join(t.split()[0:MAX_SEQ_LENGTH]) for t in test_text]
test_text = np.array(test_text, dtype=object)
test_label = test_df['sentiment'].tolist()

train_input_ids, train_input_masks, train_segment_ids, train_labels = tokenizer.predict(train_text, train_label, verbose=True)
test_input_ids, test_input_masks, test_segment_ids, test_labels = tokenizer.predict(test_text, test_label, verbose=True)

Converting examples to features:   0%|          | 0/50000 [00:00<?, ?it/s]

INFO:tensorflow:Writing example 0 of 50000


I0609 09:28:23.886604 139943015159680 bert_tokenizer.py:246] Writing example 0 of 50000


INFO:tensorflow:*** Example ***


I0609 09:28:23.899882 139943015159680 bert_tokenizer.py:205] *** Example ***


...

I0609 09:30:35.254751 139943015159680 bert_tokenizer.py:246] Writing example 20000 of 50000
Converting examples to features:  60%|█████▉    | 29962/50000 [00:54<00:36, 541.87it/s]

INFO:tensorflow:Writing example 30000 of 50000


I0609 09:30:53.169109 139943015159680 bert_tokenizer.py:246] Writing example 30000 of 50000
Converting examples to features:  80%|███████▉  | 39961/50000 [01:12<00:17, 564.58it/s]

INFO:tensorflow:Writing example 40000 of 50000


I0609 09:31:11.255229 139943015159680 bert_tokenizer.py:246] Writing example 40000 of 50000
Converting examples to features: 100%|██████████| 50000/50000 [01:30<00:00, 551.92it/s]


## Model

In [0]:
def build_model_bert(max_seq_length): 
    in_id = Input(shape=(max_seq_length,), name="input_ids")
    in_mask = Input(shape=(max_seq_length,), name="input_masks")
    in_segment = Input(shape=(max_seq_length,), name="segment_ids")
    bert_inputs = [in_id, in_mask, in_segment]
    
    bert_output = BertLayer(n_fine_tune_layers=2, tf_hub=BERT_PATH, output_representation='mean_pooling', trainable=True)(bert_inputs)
    
    dense = Dense(256, activation='relu')(bert_output)
    pred = Dense(1, activation='sigmoid')(dense)
    
    model = Model(inputs=bert_inputs, outputs=pred)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.summary()
    
    return model

def initialize_vars(sess):
    sess.run(tf.local_variables_initializer())
    sess.run(tf.global_variables_initializer())
    sess.run(tf.tables_initializer())
    K.set_session(sess)

In [19]:
bert_model = build_model_bert(MAX_SEQ_LENGTH)
initialize_vars(sess)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I0609 09:31:35.717531 139943015159680 saver.py:1483] Saver not created because there are no variables in the graph to restore


__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_ids (InputLayer)          (None, 256)          0                                            
__________________________________________________________________________________________________
input_masks (InputLayer)        (None, 256)          0                                            
__________________________________________________________________________________________________
segment_ids (InputLayer)        (None, 256)          0                                            
__________________________________________________________________________________________________
bert_layer_1 (BertLayer)        (None, 768)          110104890   input_ids[0][0]                  
                                                                 input_masks[0][0]                
          

In [20]:
bert_model.fit([train_input_ids, train_input_masks, train_segment_ids], 
               train_labels,
               validation_data=([test_input_ids, test_input_masks, test_segment_ids], test_labels),
               epochs=1,
               batch_size=64)

Train on 50000 samples, validate on 50000 samples
Epoch 1/1
50000/50000 [==============================] - 1911s 38ms/step - loss: 0.5329 - acc: 0.6739 - val_loss: 0.5125 - val_acc: 0.6805


In [0]:
bert_model.save('bert.h5')